# Data Scraping
## This file contains the code to access the public information of a popular Twitter user to build the required dataset

In [1]:
#Importing necessary packages
import tweepy
import sqlite3

In [ ]:
#creating a tweepy object to access the twitter API
consumer_key = 'tHaeI3Vfg3B5BxDdAEnmb5qHp'
consumer_secret = 'bazSR24b793Th2lByeyc7bA319ZTWnx7CvwMVzVzhuCERQbjTn'

access_token = '1140113387952586752-JUA8pQaeQp6z6inciK8TjuqsxwwxL1'
access_token_secret = 'i4stpoFuZJQNMY8xiM9FNpyT02j70lSyiJhL0yChjG7Jd'

auth = tweepy.OAuthHandler (consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
#accessing the information of a popular twitter user which huge count of followers. Then accessing the information of first
#5000 followers of the popular user
#Replace '----' with any popular twitter user (no particular username specified due to security reasons)
mr_popular = api.get_user(screen_name = '----')
#print (type(mr_popular))
#print (mr_popular)

followers_mr_popular = api.followers_ids('----')
#print (type(followers_mr_popular))
#print (len(followers_mr_popular))

In [ ]:
#creating a sqlite database to save the information
conn = sqlite3.connect('twitter_followers.db')
print ('Opened database succesfully')

#creating a new table
conn.execute ('''create table twitter_data1(
        user_id int primary key not null,
        user_screenname text not null,
        user_name text not null,
        user_location text,
        user_bio text,
        user_createdat date,
        user_followers_count int,
        user_friends_count int,
        user_favourites_count int,
        user_laststatus_date date,
        user_laststatus_text text,
        user_statuses_count int);''')

In [ ]:
#THE MAIN LOOP: In this loop, one by one each of the popular user's follower's information is extracted and saved(inserted)
#in the newly created table.
#IMPORTANT: accessing more than 1000 users at an instant results in a 'rate-limit-exceeded' error by the API, so the script 
#is to be run accordingly if more than a 1000 users are to be accessed.

for i, follower in enumerate(followers_mr_popular[0:1000]):
    try:
        follower_id = api.get_user(user_id = follower)
        is_protected = follower_id.protected
        s_c = follower_id.statuses_count
        if not is_protected and s_c:
            fuser_id = follower_id.id
            fuser_screen_name = follower_id.screen_name
            fuser_name = follower_id.name
            fuser_location = follower_id.location
            fuser_desc = follower_id.description
            dfuser_created_at = follower_id.created_at
            fuser_followc = follower_id.followers_count
            fuser_friendsc = follower_id.friends_count
            fuser_favouritesc = follower_id.favourites_count
            dfuser_status_date = follower_id.status.created_at
            fuser_status_text = follower_id.status.text
            fuser_statusesc = follower_id.statuses_count
            #fuser_follows_mrpopular = int (follower_id)
            
            fuser_created_at = dfuser_created_at.strftime('%m-%d-%Y')
            fuser_status_date = dfuser_status_date.strftime('%m-%d-%Y')
            
            conn.execute('''insert or replace into twitter_data1(
                    user_id,
                    user_screenname,
                    user_name,
                    user_location,
                    user_bio,
                    user_createdat,
                    user_followers_count,
                    user_friends_count,
                    user_favourites_count,
                    user_laststatus_date,
                    user_laststatus_text,
                    user_statuses_count
                    )values(?,?,?,?,?,?,?,?,?,?,?,?)''',
                            (fuser_id,
                            fuser_screen_name,
                            fuser_name,
                            fuser_location,
                            fuser_desc,
                            fuser_created_at,
                            fuser_followc,
                            fuser_friendsc,
                            fuser_favouritesc,
                            fuser_status_date,
                            fuser_status_text,
                            fuser_statusesc))
            conn.commit()
    
    except tweepy.TweepError as e:
        print (e)
        continue

In [ ]:
#debugging code to keep track of the no. of users accessed
cursor = conn.execute('''select count(*) from twitter_data1;''')
for row in cursor:
    print ('count = ', row[0])

In [ ]:
#FOLLOWING TWITTER USERS BASED ON MUTUAL INTERESTS: each of the user from the database is compared against certain criterias
#and consecutively followed
users = []
cursor = conn.execute('''select * from twitter_data1;''')
for row in cursor:
    users.append(row[0])
print (len(users))

#The following list contains the keywords that is used to target users with mutual interests (as per their bio)
interests = ['machine', 'learning', 'computer', 'science', 'programming', 'AI', 
             'artificial', 'intelligence', 'deep', 'nlp', 'processing', 'data', 
             'cs', 'statistics', 'statistic', 'maths', 'mathematics', 'scientist',
             'programmer', 'computing']

#IMPORTANT: Twitter does not permit following more than 400 people in 24 hrs. The code needs to be adjusted accordingly.
for k in users[0:300]:
    try:
        follower1 = api.get_user(id = k)
        is_protected1 = follower1.protected
        status_count1 = follower1.statuses_count
        
        if not is_protected1 and status_count1:
            follower_bio = follower1.description.lower()
            follower_favourites_count = follower1.favourites_count
            follower_friends_count = follower1.friends_count
            dlast_status = follower1.status.created_at
            last_status = dlast_status.strftime('%m-%d-%Y')   
            if (last_status > '04-01-2019'):
                is_active = 1
            else:
                is_active = 0
                
            if any(interest in follower_bio for interest in interests) \
                            and follower_favourites_count > 20 \
                            and follower_friends_count > 100:
                                print ('case is working')
                                api.create_friendship(id = follower1.id)
                                print ('doing friendhsip....')
                                save_id(follower1.id)
    
    except tweepy.TweepError as ec:
        print (ec)
        continue 

In [ ]:
#The same scipt was used against another popular user to bring in diversity in terms of the people followed